# Multipool Tribrata News Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-tbn.sh >> /home/haviz/multipool/ai-sa-tbn.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '103.82.242.92',
        'port': '5710',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData(bind=engine)

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 

    # we calculate sentiment for 'title' and 'description' column

    # remove first
    source_ds['title'] = source_ds['title'].apply(clean_up_tag)
    source_ds['description'] = source_ds['content'].apply(clean_up_tag)


    # skip stemming
    source_ds['stemmed'] = source_ds['title']
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        res = pipe(list_text[i])
        #print(res)
        new_list.append(res)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
def process_by_df_col(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds

In [6]:
# query 400 rows of data query data from table
select_query = "select * \
from news_tbn_medol gr \
where id not in (select id_tbn_news from news_tbn_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [7]:
# Processing title sentiment analysis
content_ds = source_ds
res_ds = process_by_df(source_ds)

100%|██████████| 500/500 [00:24<00:00, 20.35it/s]


In [8]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,tbn_medol_id,is_polri,reporter,editor,description,stemmed,tokenized_tweet,Prediction,Score
0,943,Mencegah Gangguan Kamtibmas Bhabinkamtibmas De...,Mencegah Gangguan Kamtibmas Bhabinkamtibmas De...,04 February 2024 01:30:49 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.299107,17,False,,,Mencegah Gangguan Kamtibmas Bhabinkamtibmas De...,mencegah gangguan kamtibmas bhabinkamtibmas de...,"{kamtibmas, bumbung, bhabinkamtibmas, pementas...",Neutral,0.999629
1,942,Dengan Intensif Lakukan Patroli Polsek Manggis...,Polda Bali - Polres Karangasem - Polsek Manggi...,04 February 2024 05:19:59 Wita,Humas Polsek Manggis.,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.294400,17,False,,,Polda Bali Polres Karangasem Polsek Manggis Pa...,dengan intensif lakukan patroli polsek manggis...,"{kamtibmas, polsek, manggis, patroli, intensif...",Neutral,0.999643
2,941,Guna Mencptakan Keamanan Dan Kenyamanan Menjel...,Dalam mengantisipasi adanya gangguan keamanan...,04 February 2024 06:12:23 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.283713,17,False,,,Dalam mengantisipasi adanya gangguan keamanan ...,guna mencptakan keamanan dan kenyamanan menjel...,"{menjelang, laksanakan, patrol, polsek, polres...",Neutral,0.999662
3,940,Tingkatkan Kewaspaadan Dengan Sispam Mako Pers...,"Personil Polsek Sidemen Polres Karangasem, Da...",04 February 2024 06:17:38 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.268219,17,False,,,Personil Polsek Sidemen Polres Karangasem Dala...,tingkatkan kewaspaadan dengan sispam mako pers...,"{sispam, personil, jaga, sidemen, polsek, anti...",Neutral,0.999680
4,939,Pantau Anjungan ATM Serta Sambangi Obyek Vital,Guna menciptakan rasa aman dan nyaman masyara...,04 February 2024 06:20:03 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.263906,17,False,,,Guna menciptakan rasa aman dan nyaman masyarak...,pantau anjungan atm serta sambangi obyek vital,"{sambangi, obyek, pantau, atm, anjungan, vital}",Neutral,0.999759


In [9]:
# Processing content sentiment analysis
res_content_ds = process_by_df_col(content_ds,'content')

100%|██████████| 500/500 [03:13<00:00,  2.58it/s]


In [10]:
res_content_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,tbn_medol_id,is_polri,reporter,editor,description,stemmed,tokenized_tweet,Prediction,Score
0,943,Mencegah Gangguan Kamtibmas Bhabinkamtibmas De...,Mencegah Gangguan Kamtibmas Bhabinkamtibmas De...,04 February 2024 01:30:49 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.299107,17,False,,,Mencegah Gangguan Kamtibmas Bhabinkamtibmas De...,mencegah gangguan kamtibmas bhabinkamtibmas de...,"{sambang, banjar, anak, joged, teges, membayar...",Positive,0.701932
1,942,Dengan Intensif Lakukan Patroli Polsek Manggis...,Polda Bali - Polres Karangasem - Polsek Manggi...,04 February 2024 05:19:59 Wita,Humas Polsek Manggis.,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.294400,17,False,,,Polda Bali Polres Karangasem Polsek Manggis Pa...,polda bali polres karangasem polsek manggis pa...,"{lancar, arus, landai, pkl, masyarakat, manggi...",Neutral,0.989013
2,941,Guna Mencptakan Keamanan Dan Kenyamanan Menjel...,Dalam mengantisipasi adanya gangguan keamanan...,04 February 2024 06:12:23 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.283713,17,False,,,Dalam mengantisipasi adanya gangguan keamanan ...,dalam mengantisipasi adanya gangguan keamanan ...,"{menjelang, wayan, s, m, menjaga, ketertiban, ...",Neutral,0.967124
3,940,Tingkatkan Kewaspaadan Dengan Sispam Mako Pers...,"Personil Polsek Sidemen Polres Karangasem, Da...",04 February 2024 06:17:38 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.268219,17,False,,,Personil Polsek Sidemen Polres Karangasem Dala...,personil polsek sidemen polres karangasem dala...,"{sabtu, wayan, s, penjagaan, m, menjaga, polis...",Neutral,0.973815
4,939,Pantau Anjungan ATM Serta Sambangi Obyek Vital,Guna menciptakan rasa aman dan nyaman masyara...,04 February 2024 06:20:03 Wita,,tirto,https://tribratanews.bali.polri.go.id/artikel/...,2024-02-04 01:56:48.263906,17,False,,,Guna menciptakan rasa aman dan nyaman masyarak...,guna menciptakan rasa aman dan nyaman masyarak...,"{pelaku, sabtu, pemasangan, polsek, mencegah, ...",Neutral,0.991682


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    # sql = "INSERT INTO cekmedsos_database.ret_sentiment_result (job_id, tweet_id, sentiment_class) VALUES(%s, %s , %s);" % (str(const_job_id), row['id_str'], str(sentiment_class))
    sql = "INSERT INTO public.news_tbn_sentiment (id_tbn_news, sentiment_title, sentiment_title_value) VALUES( %s, %s, %s);" % (row['id'], sentiment_class, row['Score'],) 
    #print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 row

In [12]:
# record result
sentiment_class = 0

for index, row in res_content_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "UPDATE public.news_tbn_sentiment set sentiment_content = %s, sentiment_content_value = %s where id_tbn_news = %s;" % (sentiment_class, row['Score'], row['id'])
    
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 row